# LOOPBACK TESTER

This notebook is useful for loopback tests when you have some Borealis data you would like to verify. This notebook is built off of the original test_beamforming.py script that I wrote in Nov/Dec 2018 to verify beamforming algorithms in Borealis. However, these analyses are useful to verify data after any update to the Borealis code. What follows is a list of functions that I plan to build into this script. - Marci Detwiller Feb 2019

**If only release-mode data is available for a given time, this notebook will:**
1. Plot the time domain data, or a portion of the data 
2. Verify location of the pulses in the data and pulse length
3. Plot the frequency spectrum and find the peaks 
4. Verify the beamforming if output_samples_iq is available

**If debug-mode data is available (rawrf, txdata), this notebook will also:**
1. Verify the rawrf from the txdata
2. Find the peaks of the FFT in the rawrf data and compare to the bfiq data to verify the decimation




In [16]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import matplotlib.pyplot as plt
import deepdish
import numpy as np
import random
sys.path.append('../testing_utils/plot_borealis_hdf5_data/')

from test_beamforming import plot_bf_iq_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
filename = '/data/borealis_data/20190206/20190206.2101.04.sas.0.bfiq.hdf5'
data_file = os.path.basename(filename)
data_directory = os.path.dirname(os.path.dirname(filename)) # get directory outside of the data (/data/borealis_data)

record_name = None  # or change if you want a specific record 

In [18]:
# get the corresponding data files to this data file, including output_samples_iq, bfiq,
# rawrf, txdata if available.

data_file_metadata = data_file.split('.')

date_of_file = data_file_metadata[0]
timestamp_of_file = '.'.join(data_file_metadata[0:3])
station_name = data_file_metadata[3]
slice_id_number = data_file_metadata[4]
type_of_file = data_file_metadata[-2]  # XX.hdf5
if type_of_file == slice_id_number:
    slice_id_number = '0'  # choose the first slice to search for other available files.
else:
    type_of_file = slice_id_number + '.' + type_of_file
file_suffix = data_file_metadata[-1]

if file_suffix != 'hdf5':
    raise Exception('Incorrect File Suffix: {}'.format(file_suffix))

output_samples_filetype = slice_id_number + ".output_samples_iq"
bfiq_filetype = slice_id_number + ".bfiq"
rawrf_filetype = "rawrf"
tx_filetype = "txdata"
file_types_avail = [bfiq_filetype, output_samples_filetype, tx_filetype, rawrf_filetype]

if type_of_file not in file_types_avail:
    raise Exception(
        'Data type: {} not incorporated in script. Allowed types: {}'.format(type_of_file,
                                                                             file_types_avail))

data = {}
print('Available Filetypes: ')
for file_type in list(file_types_avail):  # copy of file_types_avail so we can modify it within.
    try:
        filename = data_directory + '/' + date_of_file + '/' + timestamp_of_file + \
                    '.' + station_name + '.' + file_type + '.hdf5'
        data[file_type] = deepdish.io.load(filename)
        print(file_type)
    except:
        file_types_avail.remove(file_type)
        if file_type == type_of_file:  # if this is the filename you provided.
            raise

Available Filetypes: 
0.bfiq
0.output_samples_iq


In [20]:
# Choose a record from the provided file, and get that record for each filetype to analyze side by side.
# Also reshaping data to correct dimensions - if there is a problem with reshaping, we will also not use that record.

good_record_found = False
record_attempts = 0
while not good_record_found:
    if record_name is None:
        record_name = random.choice(list(data[type_of_file].keys()))
    print('Record Name Chosen: {}'.format(record_name))

    record_data = {}

    try:
        for file_type in file_types_avail:
            record_data[file_type] = data[file_type][record_name]

            if file_type == bfiq_filetype:
                bf_iq = record_data[bfiq_filetype]
                number_of_beams = len(bf_iq['beam_azms'])
                number_of_arrays = len(bf_iq['antenna_arrays_order'])
                flat_data = np.array(bf_iq['data'])  
                # reshape to 2 (main, intf) x nave x number_of_beams x number_of_samples
                bf_iq_data = np.reshape(flat_data, (number_of_arrays, bf_iq['num_sequences'], number_of_beams, bf_iq['num_samps']))
                bf_iq['data'] = bf_iq_data
                beam_azms = bf_iq['beam_azms']
                pulses = bf_iq['pulses']
                decimated_rate = bf_iq['rx_sample_rate']
                tau_spacing = bf_iq['tau_spacing']
                freq = bf_iq['freq']
                nave = bf_iq['num_sequences']
                main_antenna_count = bf_iq['main_antenna_count']
                intf_antenna_count = bf_iq['intf_antenna_count']

            if file_type == output_samples_filetype:
                output_samples_iq = record_data[output_samples_filetype]
                number_of_antennas = len(output_samples_iq['antenna_arrays_order'])

                flat_data = np.array(output_samples_iq['data'])  
                # reshape to number of antennas (M0..... I3) x nave x number_of_samples
                output_samples_iq_data = np.reshape(flat_data, (number_of_antennas, output_samples_iq['num_sequences'], output_samples_iq['num_samps']))
                output_samples_iq['data'] = output_samples_iq_data
                antennas_present = [int(i.split('_')[-1]) for i in output_samples_iq['antenna_arrays_order']]
                output_samples_iq['antennas_present'] = antennas_present

            if file_type == rawrf_filetype:
                rawrf = record_data[rawrf_filetype]
                number_of_antennas = rawrf['main_antenna_count'] + rawrf['intf_antenna_count']
                #number_of_antennas = len(rawrf['antenna_arrays_order'])
                flat_data = np.array(rawrf['data'])  
                # reshape to num_sequences x number_of_antennas x number_of_samples
                rawrf_data = np.reshape(flat_data, (rawrf['num_sequences'], number_of_antennas, rawrf['num_samps']))
                rawrf['data'] = rawrf_data
                rawrf['antennas_present'] = range(0,rawrf['main_antenna_count'] + rawrf['intf_antenna_count'])
                # these are based on filter size. TODO test with modified filter sizes and
                # build this based on filter size.

                # determined by : 0.5 * filter_3_num_taps * dm_rate_1 * dm_rate_2 + 0.5 *
                # filter_3_num_taps. First term is indicative of the number of samples
                # that were added on so that we don't miss the first pulse, second term
                # aligns the filter so that the largest part of it (centre) is over the pulse.

                # This needs to be tested.
                rawrf['dm_start_sample'] = 180*10*5 + 180

            # tx data does not need to be reshaped.
            if file_type == tx_filetype:
                tx = record_data[tx_filetype]
                tx['rx_sample_rate'] = tx['tx_rate'][0]/tx['dm_rate']
                print('Decimation rate error: {}'.format(tx['dm_rate_error']))
                print(tx['rx_sample_rate'])
                tx['data_descriptors'] = ['num_sequences', 'num_antennas', 'num_samps']
                tx['data'] = tx['decimated_tx_samples']
                tx['antennas_present'] = tx['decimated_tx_antennas'][0]
                tx['dm_start_sample'] = 0


    except ValueError as e:
        print('Record {} raised an exception in filetype {}:\n'.format(record_name, file_type))
        traceback.print_exc()
        print('\nA new record will be selected.')
        record_attempts +=1
        if record_attempts == 3:
            print('FILES FAILED WITH 3 FAILED ATTEMPTS TO LOAD RECORDS.')
            raise # something is wrong with the files 
    else:  # no errors
        good_record_found = True

if bfiq_filetype not in file_types_avail:
    raise Exception('BFIQ data is required to do tests and could not be loaded.')

Record Name Chosen: 1549486913436


In [ ]:
    # find pulse points in data that is decimated. 

    #plot_output_samples_iq_data(record_data[output_samples_filetype], output_samples_filetype)
    plot_bf_iq_data(record_data[bfiq_filetype], bfiq_filetype)
    #plot_antenna_data(record_data[tx_filetype]['tx_samples'][0,:,:], record_data[tx_filetype]['tx_antennas'][0], tx_filetype, main_antenna_count)
    #plot_antenna_data(record_data[output_samples_filetype]['data'][:,0,:], record_data[output_samples_filetype]['antenna_arrays_order'], output_samples_filetype, main_antenna_count, separate_plots=False)
    #plot_antenna_data(record_data[rawrf_filetype]['data'][0,:,18000:20000], record_data[rawrf_filetype]['antennas_present'], rawrf_filetype, main_antenna_count,separate_plots=False)
    #for antenna in range(0,record_data[tx_filetype]['tx_samples'].shape[1]):
    #    fft_and_plot(record_data[tx_filetype]['tx_samples'][0,antenna,:], 5000000.0)